# Plot creation

In [ ]:
from pathlib import Path

In [ ]:
root = Path('/Users/klay6683/Dropbox/data/ciss/SOI')

In [ ]:
list(root.glob('*.cub'))

In [ ]:
res = glob.glob('/cassini_ringsdata/opus/N*/*.map.dst.cal.cub')

In [ ]:
fnames = pd.Series(res)
fnames.head()

In [ ]:
fnames.size

In [ ]:
fnames = pd.DataFrame(fnames)

In [ ]:
fnames.columns = ['filtered']

In [ ]:
fnames.head()

In [ ]:
import os
fnames['obs_id'] = fnames.filtered.map(lambda x: os.path.dirname(x).split('/')[-1])

In [ ]:
fnames.head()

In [ ]:
fnames.set_index('obs_id', inplace=True)

In [ ]:
fnames.head()

In [ ]:
def get_unfiltered_from_filtered(filtered):
    dirname = os.path.dirname(filtered)
    unfiltered = glob.glob(os.path.join(dirname, '*.map.cal.cub'))
    if unfiltered:
        return unfiltered[0]
    else:
        return None
fnames['unfiltered']=fnames.filtered.map(get_unfiltered_from_filtered)

In [ ]:
fnames.describe()

In [ ]:
SOI = fnames[fnames.filtered.str.contains('N146734')]

In [ ]:
SOI

In [ ]:
def convert_to_png(fname):
    import gdal
    from pyciss import io
    ds = gdal.Open(fname)
    if ds.RasterXSize > 5200:
        return fname, False
    cube = io.RingCube.open(fname)
    cube.imshow(save=True, extra_title='Filtered')
    return fname, True

In [ ]:
from IPython.parallel import Client
c = Client()

In [ ]:
lbview = c.load_balanced_view()

In [ ]:
results = lbview.map_async(convert_to_png, SOI.filtered)

In [ ]:
from iuvs.multitools import *

In [ ]:
from time import sleep
prog = IntProgress(min=0, max=len(SOI))
display(prog)
while not results.ready():
    sleep(1)
    prog.value = results.progress
print("Done.")

In [ ]:
results.result

In [ ]:
for fname in SOI.filtered:
    dirname = os.path.dirname(fname)
    sourcename = dirname+'/*.png'
    source2name = dirname+'/*.map.dst.cal.cub'
    targetname = '/cassini_ringsdata/SOI/'
    print(sourcename, source2name, targetname)
    !cp {sourcename} {source2name} {targetname}

# Comparison

In [ ]:
plt.ioff()
import os
for fname in SOI:
    print(os.path.basename(fname))
    fig, axes = plt.subplots(ncols=2, figsize=io.calc_4_3(10))
    oldcube = io.RingCube.open(fname.split('.')[0]+'.map.cal.cub')
    oldcube.imshow(ax=axes[0])
    newcube = io.RingCube.open(fname)
    newcube.imshow(ax=axes[1])
    savename = os.path.basename(fname).split('.')[0]+'_compare.png'
    fig.savefig('/cassini_ringsdata/SOI/'+savename, dpi=150)

# Create tifs

In [ ]:
from pysis.isis import isis2std
for fname in SOI.filtered:
    dirname = os.path.dirname(fname)
    cube = glob.glob(os.path.join(dirname, '*.map.dst.cal.cub'))[0]
    print(cube)
    isis2std(from_=cube, to=cube[:-3]+'png', format='png',
             minpercent=2, maxpercent=98,)
#     bittype='8bit')